In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

Before dig into our first have something about the problem.Here our objective is to predicting the churning of people from this company in future . And at the end Compare the prediction between Deep learning model and Machine learning model, how they work corresponding to this dataset.



Content of this kernel.
* Collecting the Data.
* Analyzing the data.
* Wrangling the data.
* Fature Engineering.
* Make prediction with the help of ANN

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras

#Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV

%matplotlib inline

In [ ]:
df = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
#Let's check the feature typs
df.dtypes

Here we are seeing that 3 features are Numerical and rest all of are strings type.But here one noticable point is 
'TotalCharges' feature is given as object type, which cann,t be object and we have to convert it into numeric type. 


In [ ]:
df.TotalCharges.values

In [ ]:
pd.to_numeric(df.TotalCharges)

In [ ]:
#Here we are seeing that some of the valus contain space(' ') let's tackle it.
pd.to_numeric(df.TotalCharges, errors = 'coerce')

In [ ]:
#Let's have a glance of those rows which contains space (' ').
df[pd.to_numeric(df.TotalCharges, errors = 'coerce').isnull()]

 Here we are seeing that out of 7043, 11 rows contains ' '.So we can drop those rows .

In [ ]:
new_df = df[df.TotalCharges != ' ']

In [ ]:
#So now safely convert the feature type.
new_df.TotalCharges = pd.to_numeric(new_df.TotalCharges)

In [ ]:
new_df.TotalCharges.dtypes

# EDA

In [ ]:
sns.countplot(x = 'Churn', data = new_df)

In [ ]:
#Let'us see the realationship between numercal features and the target features.
tenure_churn_yes = new_df[new_df.Churn=='Yes'].tenure
tenure_churn_no = new_df[new_df.Churn=='No'].tenure

plt.hist([tenure_churn_yes, tenure_churn_no], color = ['green', 'red'], label = ['Churn = Yes', 'Churn = No'] )
plt.legend()

From the above chart we are seeing that man who are associated with the company greater than 60 months are likly to not churning.
And who are less then 10 months are likely to churn the company.

In [ ]:
mc_churn_yes = new_df[new_df.Churn=='Yes'].MonthlyCharges
mc_churn_no = new_df[new_df.Churn=='No'].MonthlyCharges

plt.hist([mc_churn_yes, mc_churn_no], color = ['green', 'red'], label = ['Churn = Yes', 'Churn = No'] )
plt.legend()

Here we are seeing that more 1400 people are not likely to churn the company as they have monthlychargers less then 30

In [ ]:
sns.countplot(x = 'gender', hue = 'Churn', data =new_df)

# Data Cleaning

In [ ]:
new_df.isnull().sum()

Outliers Checking:

In [ ]:
#Out_check = ( 'tenure', 'MonthlyCharges','TotalCharges')
sns.boxplot(x = 'Churn', y = 'tenure', data = new_df)

In [ ]:
sns.boxplot(x = 'Churn', y = 'MonthlyCharges', data = new_df)

In [ ]:
sns.boxplot(x = 'Churn', y = 'TotalCharges', data = new_df)

Here we are seeing there is decent amount of outliers in 'TotalCharges' features.And i think it willnot
be wise delete or substitue those value because this feature obtained from multiplyinf 'MonthlyCharges' by 'tenure'
and which is not imaginary but true values.

In [ ]:
# I dont't think customerID columns will be eventfull in our way so drop it
new_df.drop(['customerID'], axis = 1, inplace = True)

In [ ]:
def uni_col_val(df):
    for column in df:
        if df[column].dtype == 'object':
            print(f'{column} : {df[column].unique()}')

In [ ]:
uni_col_val(new_df)

Here we are seeing that a lot's of column contain yes and no value.And 'No phone service and 'No internet service' is equavalent to no.So first convert those values into no. 

In [ ]:
new_df.replace( 'No phone service', 'No', inplace = True )
new_df.replace( 'No internet service', 'No', inplace = True )

In [ ]:
uni_col_val(new_df)

Here we are seeing some categorical features we will handlee this but before that let's map the rest all the features.

In [ ]:
new_df['Partner'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['Dependents'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['PhoneService'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['MultipleLines'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['OnlineSecurity'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['OnlineBackup'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['DeviceProtection'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['TechSupport'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['StreamingTV'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['StreamingMovies'].replace({'Yes' : 1, 'No' : 0},inplace = True)
new_df['Churn'].replace({'Yes' : 1, 'No' : 0},inplace = True)  
new_df['PaperlessBilling'].replace({'Yes' : 1, 'No' : 0},inplace = True)
           

In [ ]:
new_df['gender'].replace({'Female':1, 'Male':0},inplace = True)

In [ ]:
for col in new_df:
    print(f'{col} : {new_df[col].unique()}')
    

In [ ]:
#new_df['gender'].replace({'Female':1, 'Male':0},inplace = True)

In [ ]:
new_df.head()

In [ ]:
#Let's do label_encoding for the rest two categorical_features
le_IS = LabelEncoder()
le_CT = LabelEncoder()
le_PM = LabelEncoder()

In [ ]:
new_df['InternetService'] = le_IS.fit_transform(new_df['InternetService'])
new_df['Contract'] = le_CT.fit_transform(new_df['Contract'])
new_df['PaymentMethod'] =le_PM.fit_transform(new_df['PaymentMethod'])


Now our data is totally cleaned up and try to find some insights:

In [ ]:
# first have the corelation
#colormap = plt.cm.RdBu
plt.figure(figsize=(20,20))
plt.title('Person Correlation of Features', y=1.05, size=15)
sns.heatmap(new_df.corr(),linewidths=0.1,vmax=1.0, 
            square=True,  linecolor='white', annot=True)

Here we are not seeing a conclusive relationship among the features except Monthly and Total charges.Though i will try to get the best out of it. 

In [ ]:
sns.relplot(x= 'TotalCharges', y = 'tenure' , hue = 'Churn', data = new_df)

In [ ]:
sns.relplot(x= 'TotalCharges', y = 'MultipleLines' , hue = 'Churn', data = new_df)

Except few people around 4000 rest of the are having Multiple line = 0 are not interested in Churn.
And people having multiple line and totalcharges greater then 6000 are also not interested in Churn.

In [ ]:
sns.relplot(x= 'TotalCharges', y = 'OnlineBackup' , hue = 'Churn', data = new_df)

And people having OnlineBackup and totalcharges greater then 4000 are not interested in Churn.

In [ ]:
sns.relplot(x= 'TotalCharges', y = 'DeviceProtection' , hue = 'Churn', data = new_df)

In [ ]:
sns.relplot(x= 'TotalCharges', y = 'StreamingTV' , hue = 'Churn', data = new_df)

People who's charges greater then  4000 rest of the are having Streaming TV = 0 are totally  not interested in Churn.


In [ ]:
sns.relplot(x= 'TotalCharges', y = 'StreamingMovies' , hue = 'Churn', data = new_df)

Let's try to find some insight by seeing the corelation

In [ ]:
sns.relplot(x= 'Dependents', y = 'Partner' , hue = 'Churn', data = new_df)

This giving us an clear indication that Person having partner and Dependents == 0 are  only interested in Churning.

In [ ]:
sns.relplot(x= 'InternetService', y = 'PhoneService' , hue = 'Churn', data = new_df)

Till now we have seen realtionship among features wirh respect to target.

Now let's how does all the features related to the target. 

In [ ]:
new_df.head()

In [ ]:
features = new_df[['gender','SeniorCitizen','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod']].columns

In [ ]:
for i in features:
    sns.countplot(x = i, hue = 'Churn', data = new_df)
    plt.show()

In [ ]:
#Let's scale down the data.

cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']
scaler = MinMaxScaler()
new_df[cols_to_scale] = scaler.fit_transform(new_df[cols_to_scale])

In [ ]:
new_df.head()

In [ ]:
X = new_df.drop(['Churn'], axis=1)
y = new_df.Churn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:


len(X_train.columns)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(20, input_shape = (19,),activation = 'sigmoid'),
     keras.layers.Dense(16,activation = 'relu' ),
     keras.layers.Dense(12,activation = 'relu' ),
     keras.layers.Dense(1,activation = 'sigmoid' )
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
    
)

In [ ]:
model.fit(X_train, y_train, epochs = 20)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_p = model.predict(X_test)


In [ ]:
y_p[:10]

In [ ]:
y_pred = []
for output in y_p:
    if output>.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
    
    

In [ ]:
y_pred[:10]

In [ ]:
y_test[:10]

Reporting the result:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
 print(classification_report(y_test,y_pred))

In [ ]:
cm = tf.math.confusion_matrix(labels = y_test, predictions = y_pred)
sns.heatmap(cm,annot = True,fmt = 'd')

Let's try to understand how confusion_matrix is formatted.

In [ ]:

#Accuracy
round((916+196)/(916+117+178+196),2)

Precision for 0 class i.e Precision for customers who didn't chuened. 

In [ ]:
round(916/(916+178),2)

Precision for 1 class i.e Precision for customers who actually  churned.

In [ ]:
round(196/(196+117),2)

Recall for 0 class

In [ ]:
round(916/(916+117), 2)

Recall for 1 class

In [ ]:
round(196/(178+196), 2)

### Now let's try with Machine learning Algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
print('Mean Squared Error is', mean_squared_error(y_test, y_pred))


# finding perfect k Value.

In [ ]:
Error = []

for i in range(1, 51):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    Error.append(mean_squared_error(y_test,y_pred))

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(1,51), Error, color = 'blue', marker = 'o', markerfacecolor = 'red', markersize = 10)

plt.title('Error rate vs K. value')
plt.xlabel('k')
plt.ylabel('Error Rate')

here we are seeing that around 50error is less and in a stable mood.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print('Mean Squared Error is', mean_squared_error(y_test, y_pred))


# Cross Validation

In [ ]:
cv = KFold(n_splits = 6, random_state = 0, shuffle = True)

Error = []

for i in range(1, 51):
    knn = make_pipeline(StandardScaler(),KNeighborsClassifier(n_neighbors=i)) 
    y_pred = cross_val_predict(knn, X,y, cv = cv)
    Error.append(mean_squared_error(y,y_pred))


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(1,51), Error, color = 'red', marker = 'o', markerfacecolor = 'blue', markersize = 10)

plt.title('Error rate vs K. value')
plt.xlabel('k')
plt.ylabel('Error Rate')